In [ ]:
#Importing the packages to run on a real quantum device
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager 
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import EstimatorOptions

#This code runs the CHSH game with a quantum strategy

#Create a variable to store the number of games to be played
num_games_q = 10

#Create a variable to store the number of games won
num_wins_q = 0

#Create a progress bar to show how many games you've played
response_progress_q = widg.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Games Played:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'green'},
    orientation='horizontal'
)

display(response_progress_q)

#Using ibm_brisbane 
service = QiskitRuntimeService(channel="ibm_quantum", token="token_from_ibm")
backend = service.backend(name='ibm_brisbane')
print("Done getting the backend")


#backend = QiskitRuntimeService()
pass_manager = generate_preset_pass_manager(backend=backend)

circ_transpiled = pass_manager.run([circ])


sampler = Sampler(mode=backend)
options = EstimatorOptions()
options.resilience_level = 1
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"



#Loop through the number of games to be played
for i in range(num_games_q):
    
    #Create a quantum register, specifying how many qubits will be used in the circuit
    reg_q = QuantumRegister(2, name = "q")
    
    #Create a classical register, specifying how many classical bits will be used in the circuit
    reg_c = ClassicalRegister(2, name = "c")
    
    #Create the complete circuit, composed of the two registers
    circ = QuantumCircuit(reg_q, reg_c, name = "circ")
    
    #There are two steps in the typical process of creating an entangled state:
    #First, create a Hadamard gate on one of the qubits
    circ.h(reg_q[0])
    
    #Second, create a CNOT gate from the qubit with the H gate to the other
    circ.cx(reg_q[0], reg_q[1])
    
    #Create a variable for the number alice and bob are given, either 0 or 1
    alice_input = random.randint(0, 1)
    bob_input = random.randint(0, 1)
    
    #Your partner creates a gate based on the predetermined strategy
    if (alice_input == 0):
        
        #Does nothing and goes straight to measurement if given 0
        pass
    else:
        #Creates a hadamard gate on the first quantum register if given 1
        circ.h(reg_q[0])
    
    #Measure your partner's qubit and store the result in a classical bit
    circ.measure(reg_q[0], reg_c[0])
    
    #Create variables to store the angles the user may use
    angle_0 = np.pi/8
    angle_1 = -np.pi/8
    
    #Prompt the user to confirm their choice
    print("You received a {}. Choose which gate to apply.". format(str(bob_input)), flush = True)
    
    #Create a button widget to prompt the user for which gate to apply
    gate_button = widg.RadioButtons(
        options=[("Apply the Gate for 0", 0), ("Apply the Gate for 1", 1), ("Just Measure", 2)],
        value = 0,
        description = "Choose Which Gate to Apply",
        disabled = False
    )
        
    gate_button.layout.object_position = "bottom"
    
    display(gate_button)
    
    input("Press enter once you have made your choice")
    
    gate_chosen = gate_button.value
        
    #Apply the gate chosen by the user
    if (gate_chosen == 0):
        #Create the unitary gate, tuned to the angle pi/8
        user_gate = np.array([[np.cos(angle_0), np.sin(angle_0)], 
                               [np.sin(angle_0), -np.cos(angle_0)]])
            
        #Apply the unitary gate to the shared qubit
        circ.unitary(user_gate, range(1))
            
    elif (gate_chosen == 1):
        #Create the unitary gate, tuned to the angle -pi/8
        user_gate = np.array([[np.cos(angle_1), np.sin(angle_1)], 
                               [np.sin(angle_1), -np.cos(angle_1)]])
            
        #Apply the unitary gate to the shared qubit
        circ.unitary(user_gate, range(1))
    
    #Measure the user's qubit and store the result in a classical bit
    circ.measure(reg_q[1], reg_c[1])
        
    #Run the samppler to get counts
    sampler = Sampler(mode=backend)
    job = sampler.run([circ_transpiled])
    results = job.result()[0]
    #print(f"Sampler job ID: {job.job_id()}")
    print(f"Counts: {results.data.c.get_counts()}")

    
    #Set alice and bob's responses based on the outcome
    alice_output = int(list(outcome.keys())[0][0])
    bob_output = int(list(outcome.keys())[0][1])
        
    #Move the progress bar along to show your progress
    response_progress_q.value += 1

    #Display you and your partner's responses
    print("\nYour partner's input was {} and yours was {}.".format(str(alice_input),  str(bob_input)))
    print("Your partner returned {}  and you returned {}.".format(str(alice_output), str(bob_output)))

    #Use an 'if' statement to check whether you met the win condition
    if ((alice_output + bob_output) % 2 == alice_input * bob_input):

        #If so, increase the number of wins by one
        num_wins_q += 1
        print("You won! \n \n")

    else:
        print("You lost. \n \n")
    
#Display the number of games you and your partner won
print("You won {} out of 10 games.".format(str(num_wins_q)))
if (num_wins_q < 3):
    print("Consider trying a different strategy.")
elif (num_wins_q < 5):
    print("Better luck next time!")
elif (num_wins_q < 8):
    print("Not too shabby!")
else:
    print("You crushed it!")